In [35]:
import tushare as ts
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout,Activation

# Part 1 -model

In [ ]:
df = pd.read_csv(r'./Data/TRAINSET_STOCK.csv')
df_add = pd.read_csv(r'./Data/20190506_STOCK.csv')
df_test = pd.read_csv(r'./Data/20190514_STOCK.csv')
df_test1 = pd.read_csv(r'./Data/20190521_STOCK.csv')
df = pd.concat([df,df_add,df_test,df_test1])
df['vol_change'] = df.groupby(["ts_code"])['vol'].pct_change()
df['low_change'] = df.groupby(["ts_code"])['low'].pct_change()
df['high_change'] = df.groupby(["ts_code"])['high'].pct_change()
df['vol_change'] = df['vol_change'].fillna(0)
df['low_change'] = df['vol_change'].fillna(0)
df['high_change'] = df['vol_change'].fillna(0)
df =  df[['ts_code','trade_date','name','pct_change','vol_change','high_change','low_change','y']]
df = df[['pct_change','vol_change','high_change','low_change','y']]
dataset = df.values
x = []
y = []
for i in range(len(dataset)-15):
    x.append(dataset[i:(i+10),:5].tolist())
    y.append(dataset[(i+10):(i+15),4,np.newaxis].tolist())
xtrain, xtest, ytrain, ytest = train_test_split(
    x, y, test_size=0.2, random_state=0)
xtrain, xtest, ytrain, ytest = np.array(xtrain),np.array(xtest),np.array(ytrain),np.array(ytest)
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape
ytrain = np.reshape(ytrain, (ytrain.shape[0], ytrain.shape[1]))
ytest= np.reshape(ytest, (ytest.shape[0], ytest.shape[1]))
regressor = Sequential()
regressor.add(LSTM(units = 60, return_sequences = True, input_shape = (xtrain.shape[1], 5)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 60, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 60,return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 60))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 5))
regressor.add(Activation('sigmoid'))
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
regressor.fit(xtrain, ytrain, epochs = 100, batch_size = 50)
predicted_stock_price = regressor.predict(xtest)
a = []
for i in range(len(ytest)):
    fpr, tpr, thresholds = roc_curve(ytest[i], predicted_stock_price[i], pos_label=1)
    auc(fpr, tpr)
    a.append(auc(fpr, tpr))
a=pd.DataFrame(a)
a=a.dropna()
a.mean()

In [28]:
def pre(code):
    df1 = ts.get_hist_data(code) 
    df1=df1.head(n=11)
    df1=df1.sort_index(ascending=True)
    df1=df1[['p_change','volume','high','low']]
    df1['y']=[1,0,0,1,1,0,0,1,0,0,1]
    df1['vol_change'] = df1['volume'].pct_change()
    df1['low_change'] = df1['low'].pct_change()
    df1['high_change'] = df1['high'].pct_change()
    dataset=df1.values
    x=dataset.tolist()
    x=np.array(x)
    x=np.reshape(x, (1,x.shape[0],x.shape[1]))
    predicted = regressor.predict(x)
    print(predicted)


In [36]:
def getdata(code):
    df = ts.get_hist_data(code)
    df.to_csv('./预测.csv')

In [37]:
getdata('sh')